In [124]:
!pip install yfinance

## Stock DATASET

In [125]:
from torch.utils.data import Dataset, DataLoader
from typing import Union
import torch
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [126]:
class StockPriceDataset(Dataset):
    def __init__(self, start_date: str="jj-mm-aaaa", end_date: str="jj-mm-aaaa", 
                 interval: int=1, nb_samples: int=20, transform=None,
                 file_dir: str="data/", csv_file: str=None):

        # If a local data file must be loaded:
        if csv_file is not None:
            self.root_dir = file_dir
            self.filename = csv_file
            with open(os.path.join(file_dir,csv_file), 'r') as file:
                data = pd.read_csv(file, sep=',', header='infer')

        else: # Data must be loaded on an online database:
            dataset = yf.download('^GSPC', start=start_date, end=end_date, interval=interval)

        self.data = dataset
        self.nb_samples = nb_samples
        self.transform = transform

    def __len__(self) -> int:
        return len(self.data)//self.nb_samples

    def __getitem__(self, index) -> Union[torch.Tensor, float]:
        # Load one sample more than nb_samples for normalizing, transform
        sample = self.data['Close'][index*self.nb_samples:(index+1)*self.nb_samples+1]
        # sample = self.data['Close'][index:index+self.nb_samples+1]
        sample = torch.tensor(sample)
        if self.transform:
            sample = self.transform(sample)[1:]
        else:
            sample = sample[1:]
        
        label = sample[-1] # label is the last elem of sample

        sample = sample[:-1] # removes label from sample
        return sample, label

def normalize_by_last_unknown_price(sample: torch.Tensor) -> torch.Tensor:
    """Divides the whole stock price sample by the last unknown price w_{p*t-1}"""
    last_price = sample[0] # w_{pt-1}
    return sample/last_price


if __name__ == "__main__":

    # download S&P data from yahoo finance
    START_DATE = '1950-01-03'
    END_DATE = '2021-11-16'
    INTERVAL = '1d'
    nb_samples = 15

    dataset = StockPriceDataset(START_DATE, END_DATE, INTERVAL, nb_samples,
                                transform=normalize_by_last_unknown_price)

    dataloader = DataLoader(dataset, batch_size=16)
    print(len(dataloader))
    # for i_batch, batch in enumerate(dataloader):
    #     print("i_batch = {}, batch = {}".format(i_batch, batch))


[*********************100%***********************]  1 of 1 completed
76


## Modèle

In [127]:
from torch.nn import LSTM, Module, Dropout, ModuleList

In [128]:
class StockAI(Module):
     
    def __init__(self, input_size, lstm_size, num_layers, keep_prob) -> None:
        super().__init__()
        self.input_size = input_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.lstm = LSTM(self.input_size, hidden_size = 1, num_layers = self.num_layers, dropout = 1 - keep_prob, batch_first = True)
    
    def forward(self,x):
        a, b = self.lstm(x)
        return b[0]
    

## Configuration

In [129]:
class StockAIConfig():
    config = {
        "model":{
            "input_size": 1,
            "lstm_size": 128,
            "num_layers": 1,
            "keep_prob": 0.8
        },

        "dataset_train":{
            "start_date": '1950-01-03',
            "end_date": '2008-11-16',
            "interval_date": '1d',
            "nb_samples":15,
            "batch_size": 16,
            "shuffle":False
        },
        "dataset_test":{
            "start_date": '2008-11-17',
            "end_date": '2021-11-16',
            "interval_date": '1d',
            "nb_samples":15,
            "batch_size": 16,
            "shuffle":False
        },

        "learning":{
            "num_steps": 30,
            "init_lr": 1e-03,
            "lr_decay": 0.99,
            "init_epoch": 5,
            "max_epoch": 50
        }   
    }

##Prediction : Train / Test

In [130]:
import math
import torch
from torch.utils.data import DataLoader
from torch.nn import MSELoss
from torch.optim import RMSprop

In [131]:
# Model config
config = StockAIConfig().config

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

# Init of the Dataset_train
dataset_train = StockPriceDataset(config["dataset_train"]["start_date"], 
                            config["dataset_train"]["end_date"],
                            config["dataset_train"]["interval_date"], 
                            config["dataset_train"]["nb_samples"],
                            transform=normalize_by_last_unknown_price)

# Init dataloader of the Dataset_train
dataloader_train = DataLoader(dataset_train, config["dataset_train"]["batch_size"], config["dataset_train"]["shuffle"], drop_last=True)

# Init of the Dataset_test
dataset_test = StockPriceDataset(config["dataset_test"]["start_date"], 
                            config["dataset_test"]["end_date"],
                            config["dataset_test"]["interval_date"], 
                            config["dataset_test"]["nb_samples"],
                            transform=normalize_by_last_unknown_price)

# Init dataloader of Dataset_test
dataloader_test = DataLoader(dataset_test, config["dataset_test"]["batch_size"], config["dataset_test"]["shuffle"], drop_last=True)

# Init of the model
model = StockAI(config["model"]["input_size"],
                config["model"]["lstm_size"],
                config["model"]["num_layers"],
                config["model"]["keep_prob"])

model.to(device)

# Learning rate to use along the epochs
learning_rates = [config["learning"]["init_lr"] * (config["learning"]["lr_decay"] ** max(float(i + 1 - config["learning"]["init_epoch"]), 0.0)) for i in range(config["learning"]["max_epoch"])]

# Loss
loss_fn = MSELoss()
optimizer = RMSprop(model.parameters(), lr=learning_rates[0], eps=1e-08)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.19999999999999996 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [132]:
len(dataloader_train)

61

In [133]:
# Learning
for epoch_step in range(config["learning"]["max_epoch"]):
    lr = learning_rates[epoch_step]
    print(f"Running for epoch {epoch_step}...")
    for i_batch, batch in enumerate(dataloader_train):
        x, y = batch
        x = torch.unsqueeze(x, -1).float()
        y = y.float()
        x, y = x.to(device), y.to(device)
        y_pred = model.forward(x)
        loss = loss_fn(y_pred, y)
        #loss = torch.autograd.Variable(loss_fn(y_pred, y), requires_grad=True)

        if i_batch%10==0:
            print(f"step: {i_batch}, loss = {loss}")

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Running for epoch 0...
step: 0, loss = 0.5357494354248047
step: 10, loss = 0.39156851172447205


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([1, 16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


step: 20, loss = 0.3254278898239136
step: 30, loss = 0.2858092784881592
step: 40, loss = 0.2471795678138733
step: 50, loss = 0.222746342420578
step: 60, loss = 0.18136408925056458
Running for epoch 1...
step: 0, loss = 0.191830575466156
step: 10, loss = 0.15766489505767822
step: 20, loss = 0.13922575116157532
step: 30, loss = 0.12977278232574463
step: 40, loss = 0.11707652360200882
step: 50, loss = 0.11156065762042999
step: 60, loss = 0.09127508103847504
Running for epoch 2...
step: 0, loss = 0.09940649569034576
step: 10, loss = 0.08205941319465637
step: 20, loss = 0.07507139444351196
step: 30, loss = 0.0729711577296257
step: 40, loss = 0.06778103113174438
step: 50, loss = 0.06749594211578369
step: 60, loss = 0.05492448806762695
Running for epoch 3...
step: 0, loss = 0.06129622459411621
step: 10, loss = 0.05024588480591774
step: 20, loss = 0.04716656357049942
step: 30, loss = 0.04715589061379433
step: 40, loss = 0.044587019830942154
step: 50, loss = 0.04592587798833847
step: 60, loss =

In [134]:
#test
runnning_mape = 0
for i_batch, batch in enumerate(dataloader_test):
        x, y = batch
        x = torch.unsqueeze(x, -1).float()
        y = y.float()
        x, y = x.to(device), y.to(device)
        y_pred = model.forward(x)
        error = torch.mean(torch.abs((y - y_pred) / y))
        runnning_mape += error

mape = runnning_mape / len(dataloader_test)
print("",mape)

 tensor(0.0292, device='cuda:0', grad_fn=<DivBackward0>)
